# Calculate KAM with flexible nxn sized kernel

In [1]:
#Load packages
import numpy as np
import matplotlib.pyplot as plt

from scipy.interpolate import spline
import scipy.stats
import copy
from scipy.signal import convolve2d

import matplotlib as mpl
from skimage import morphology as mph

from quat import Quat

import ebsd
import hrdic

#Set plot behaviour. interactive grain selection has only been tested in osx display mode
%matplotlib osx

## Load data

In [2]:
mapFiles = ["./example_data_AH/ebsdData"]

maps = []

for i, mapFile in enumerate(mapFiles):
    maps.append(ebsd.Map(mapFile, "cubic"))
    maps[i].loadData(mapFile, "cubic")
    maps[i].binData = maps[i].binData[::-1]  #rotate the map 180 degrees
    maps[i].buildQuatArray()
    maps[i].findBoundaries(boundDef = 6)
    maps[i].findGrains(minGrainSize=10)
    maps[i].calcGrainMisOri(calcAxis = True)

In [3]:
maps[0].plotMisOriMap()

## Ignoring large misOris

The following function defines a NxN kernel (n), grabs the data for that kernel and finds the misorientation between the kernel centre and the rest of the kernel pixels. It then ignores any large misorientations (maxMisOri) and ignores the misorientation of the pixel with itself,and then takes the mean of the remaining misorientations in the kernel to get the local KAM. This value is stored in a KAM array, which is then plotted with overlaid grain boundaries

In [5]:
def calcKamNxN(mapData,n=3,maxMisOri=5, plotKamData=False,plotGBs=False):
    
    if n <= 1:
        raise ValueError("Please enter a kernel dimenion n, where n > 1 and n is an odd number")
    oddCheck = n%2
    if oddCheck == 0:
        raise ValueError("Please enter a kernel dimenion n, where n > 1 and n is an odd number")

    # separate the quat coefficients
    quatComps = np.empty((4, mapData.yDim, mapData.xDim))
    for i, row in enumerate(mapData.quatArray):
        for j, quat in enumerate(row):
            quatComps[:, i, j] = quat.quatCoef
    
    # define the data as the separated quat coefficients
    data = quatComps

    # define r, the remainder, to define the boundary regions
    r=int((n-1)/2)

    # kamData=np.empty((data.shape[1]-r,data.shape[2]-r))
    kamData=np.zeros((data.shape[1],data.shape[2])) # initialise with zeroes so that the final kamMap is the same size as the data

    for i in range(r,data.shape[1]-r): # avoid boundaries, r
        for j in range(r,data.shape[2]-r):

            # initialise a data kernel
            dataKernel=np.zeros((4,n,n))
            dataKernelList=np.zeros((4,n*n))

            # grab the quat data for the data kernel
            for q in range(data.shape[0]):

                # define the data kernel
                for row in range(n):
                    for column in range(n):                            
                        dataKernel[q,row,column] = data[q,i-r+row,j-r+column]
                dataKernelList[q,:]=dataKernel[q,:,:].flatten()

            #  find the mirOri between the central data pixel and the data kernel pixels
            misOriKernel=abs(np.einsum("i,ij->j",data[:,i,j],dataKernelList[:]))
            # convert to degrees
            misOriKernelDeg=2*np.arccos(misOriKernel)*180/np.pi

            # replace misOris > maxMisOri with NaN
            misOriKernelDeg_new=np.empty(n*n)
            for val in range(len(misOriKernelDeg_new)):
                if misOriKernelDeg[val] >= maxMisOri:
                    newVal=np.nan
                else:
                    newVal=misOriKernelDeg[val]
                misOriKernelDeg_new[val]=newVal

            # replace central value with NaN
            centralPixelIndex=int(((n*n)-1)/2)
            misOriKernelDeg_new[centralPixelIndex] = np.nan
            
            # find mean of all misOris in kernel
            meanMisOri=np.nanmean(misOriKernelDeg_new)    

            # place misOri in kamData
            kamData[i,j] = meanMisOri

    # Plotting
    if plotKamData:
        plt.figure()
        plt.imshow(kamData,vmin=0,vmax=1)
        plt.colorbar(shrink=0.85,label="KAM [ $^o$ ]")

        if plotGBs:
            cmap1 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap', ['black', 'black'], 256)
            cmap1._init()
            cmap1._lut[:, -1] = np.linspace(0, 1, cmap1.N + 3)

            boundaries_dilated=mph.binary_dilation(-mapData.boundaries)

            plt.imshow(boundaries_dilated, cmap=cmap1, interpolation='None', vmin=0, vmax=1)

    return kamData

In [6]:
kam3x3=calcKamNxN(maps[0],n=3,maxMisOri=5, plotKamData=True,plotGBs=True)

/Users/mclijah5/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in arccos
/Users/mclijah5/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning: Mean of empty slice


In [7]:
kam9x9=calcKamNxN(maps[0],n=9,maxMisOri=5, plotKamData=True,plotGBs=True)

/Users/mclijah5/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in arccos
/Users/mclijah5/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:49: RuntimeWarning: Mean of empty slice
